In [81]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data_loader.data_loader import PhageLoader
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [187]:
class GRU(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, output_labels, number_of_layers=1, bidirectional=True):
        super(GRU, self).__init__()
        
        self.bidirectional = bidirectional
        self.hidden_dim_dense = hidden_size
        self.num_layers = number_of_layers 
        if bidirectional:
            self.hidden_dim_dense = hidden_size * 2
            
        self.emb_layer = nn.Embedding(vocab_size, embedding_size)
        self.gru = nn.GRU(input_size=embedding_size, hidden_size=hidden_size, bidirectional=bidirectional, num_layers=number_of_layers,batch_first=True)
        self.linear = nn.Linear(self.hidden_dim_dense, output_labels)
        
        
    def forward(self, h_t1, indexes):
        #indexes -> (batch,seq_length)
        embedding = self.emb_layer(x)
        #print("EMBEDDING SHAPE: ", embedding.size())
        #embedding -> (batch,seq_length,embedding_size)
        out,h_t = self.gru(embedding, h_t1)
        #print("OUT SHAPE: ", out.size(), self.hidden_dim_dense)
        #out -> ()
        #out = out.view(self.hidden_dim_dense, -1)
        
        out = F.relu(self.linear(out))
        out2 = F.softmax(out,dim=2)
        out = F.log_softmax(out,dim=2)
        
        return out, out2,h_t
        
    def initHidden(self, batch_size, hidden_size):
        if self.bidirectional:
            return torch.zeros(self.num_layers*2, batch_size, hidden_size, device=device)
        else:
            return torch.zeros(self.num_layers, batch_size, hidden_size, device=device)

    def create_emb_layer(self, weights_matrix, non_trainable=False):
        num_embeddings, embedding_dim = weights_matrix.size()
        emb_layer = nn.Embedding(num_embeddings, embedding_dim)
        emb_layer.load_state_dict({'weight': weights_matrix})
        if non_trainable:
            emb_layer.weight.requires_grad = False

        return emb_layer, num_embeddings, embedding_dim

In [171]:

print(index.shape)
loader = PhageLoader("data/")
read_length = 100

torch.Size([1, 6])


In [239]:
model_test = GRU(vocab_size=4, embedding_size=4, hidden_size=8, output_labels=2)
n_b=2
y = torch.tensor([[1,1,1,1],[1,1,1,1]],dtype=torch.long)
y = y.type(torch.LongTensor)
x = torch.tensor([[2,1,0,3],[2,1,0,3]],dtype=torch.long)
x = x.type(torch.LongTensor)
optimizer = optim.SGD(model_test.parameters(), lr=0.01)

y = y.view(8)
hidden_test = model_test.initHidden(n_b,8)
print(hidden_test)
out,out2, hidden_test = model_test(hidden_test,x)
print(out.shape,y.shape)
l = nn.NLLLoss()
out = out.view(4*n_b,2)
print(out2)
loss = l(out,y)
print(loss)
loss.backward()
optimizer.step()
hidden_test.detach_()


hidden_test = model_test.initHidden(n_b,8)
out,out2, hidden_test = model_test(hidden_test,x)
print(hidden_test)
out = out.view(4*n_b,2)
print(out2)
loss = l(out,y)
print(loss)


loss.backward()
optimizer.step()
hidden_test.detach_()
hidden_test = model_test.initHidden(n_b,8)
out,out2, hidden_test = model_test(hidden_test,x)
out = out.view(4*n_b,2)
print(out2)
loss = l(out,y)
print(loss)


tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]])
torch.Size([2, 4, 2]) torch.Size([8])
tensor([[[0.5000, 0.5000],
         [0.4928, 0.5072],
         [0.4975, 0.5025],
         [0.4854, 0.5146]],

        [[0.5000, 0.5000],
         [0.4928, 0.5072],
         [0.4975, 0.5025],
         [0.4854, 0.5146]]], grad_fn=<SoftmaxBackward>)
tensor(0.6811, grad_fn=<NllLossBackward>)
tensor([[[ 0.2449,  0.1611,  0.5467, -0.2473,  0.0426, -0.0337,  0.0219,
           0.2608],
         [ 0.2449,  0.1611,  0.5467, -0.2473,  0.0426, -0.0337,  0.0219,
           0.2608]],

        [[ 0.1650, -0.0673, -0.4808, -0.3795, -0.3641,  0.2701, -0.0353,
           0.1670],
         [ 0.1650, -0.0673, -0.4808, -0.3795, -0.3641,  0.2701, -0.0353,
           0.1670]]], grad_fn=<StackBackward>)
tensor([[[0.5000, 0.5000],
         [0.4907, 0.5093],
         [0.4954, 0.5046],
         [0.483

In [222]:
(-np.log(0.5125) - np.log(0.5266) - np.log(0.4776) - np.log(0.5047))/4

0.6804160349145189

In [246]:
k_size=1
batch_size = 30
read_length = 100
experiment_loader = loader.get_data_loader(n_files=1,read_length=read_length, batch_size=batch_size, k=k_size, stride=1, embedding="dict", embed_size=None, drop_last=False)


In [247]:
vocab_size = 4**k_size
embedding_layer_size = 4
hidden_size = 16
output_labels=2
model = GRU(vocab_size, embedding_layer_size, hidden_size, output_labels)
print(model)
model.to(device)
hidden = model.initHidden(batch_size,hidden_size)

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

losses = []
running_loss = 0
epochs = 1
for epoch in range(epochs):
    for b, (x, y) in enumerate(experiment_loader):
        #gives batches of size "batch_size, read_length"
        
        model.zero_grad()
        x, y = x.to(device), y.to(device)
        x = x.type(torch.LongTensor)
        y = y.type(torch.LongTensor)
        if y.size()[0] != batch_size:
            break
            
        out,_,hidden = model(hidden,x)
        y = y.view(batch_size*read_length)
        out = out.view(batch_size*read_length,output_labels)
        loss = loss_function(out,y)
        running_loss = running_loss + loss.item()
        
        
        loss.backward()
        optimizer.step()
        hidden.detach_()
    print(model.emb_layer.weight)
    losses.append(running_loss)

GRU(
  (emb_layer): Embedding(4, 4)
  (gru): GRU(4, 16, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=32, out_features=2, bias=True)
)
Parameter containing:
tensor([[ 0.3619,  0.4803,  1.0733, -0.8684],
        [-0.8888, -0.0511, -0.2348, -0.1149],
        [ 0.9973,  0.0262,  1.6551,  0.8798],
        [-0.5461,  0.1211,  2.1376,  1.1199]], requires_grad=True)


In [248]:
print(model.emb_layer.weight)

Parameter containing:
tensor([[ 0.3619,  0.4803,  1.0733, -0.8684],
        [-0.8888, -0.0511, -0.2348, -0.1149],
        [ 0.9973,  0.0262,  1.6551,  0.8798],
        [-0.5461,  0.1211,  2.1376,  1.1199]], requires_grad=True)


In [269]:
first = model.emb_layer.weight[0]
second = model.emb_layer.weight[1]
third = model.emb_layer.weight[2]
fourth = model.emb_layer.weight[3]

In [259]:
print(first)

tensor([ 0.3619,  0.4803,  1.0733, -0.8684], grad_fn=<SelectBackward>)


In [275]:
cos = nn.CosineSimilarity(dim=-1,eps=1e-6)
output = cos(third, fourth)
output

tensor(0.7570, grad_fn=<DivBackward0>)

tensor(-0.3567, grad_fn=<DivBackward0>)